#### Script for graph pruning

Importing needed libraries and modules

In [1]:
import create_graph as pruning
import os
import pandas as pd

The .xlsx format is required.

In [2]:
data_address = "/Users/martinpycha/Desktop/Job_AV/metabolic_networks/data/Final_data_theoretical_lowR13_withDG.xlsx"
pruning.run_script(data_address)


PRINTING ALL THE REACTIONS
Pal.lab -> Pal, value = 168.626262497115, stat_rel = LOW

Pal.Pre -> Pal, value = 6.14492729282823e-05, stat_rel = LOW

Pal -> Pal.out, value = 75.1446429287273, stat_rel = HIGH

Glc.lab -> Glc, value = 126.735772103354, stat_rel = LOW

Glc.Pre -> Glc, value = 368.891890169068, stat_rel = LOW

Glc -> Glc.out, value = 0.627767586047383, stat_rel = LOW

Gly -> Gly.out, value = 78.1979731409389, stat_rel = HIGH

G3P -> Gly, value = 24.2070434693537, stat_rel = LOW

Glc + 2*ATP -> GAP + DAP + 2*ADP, value = 494.999894686375, stat_rel = HIGH

DAP -> GAP, value = 5.73322602406279e-05, stat_rel = LOW

DAP + 3*ATP -> G3P + 3*ADP, value = 494.999837354115, stat_rel = HIGH

GAP + 4*ADP + 2.3*dummy -> Lac + 4*ATP, value = 149.999985460193, stat_rel = LOW

GAP + 34*ADP -> CO2 + CO2 + CO2 + 34*ATP + dummy, value = 344.999966558443, stat_rel = LOW

ATP -> ADP, value = 29.060006458295, stat_rel = LOW

G3P + Pal + Pal + 2*ATP + 2*ATP -> G12PalPal + 2*ADP + 2*ADP, value = 35.

'/Users/martinpycha/Desktop/Job_AV/metabolomic_optimization'

In [ ]:
#reactions = simple_trial()

NameError: name 'simple_trial' is not defined